# 作业一：实现HMM中文分词和BPE英文分词
姓名：刘翰文

学号：522030910109

## 任务一：HMM模型用于中文分词

任务一评分标准：
1. 共有8处TODO需要填写，每个TODO计1-2分，共9分，预计代码量30行。
2. 允许自行修改、编写代码完成。对于该情况，请补充注释以便于评分，否则结果不正确将导致较多的扣分。
3. 用于说明实验的文字和总结不额外计分，但不写会导致扣分。

注：本任务仅在短句子上进行效果测试，因此对概率的计算可直接进行连乘。在实践中，常先对概率取对数，将连乘变为加法来计算，以避免出现数值溢出的情况。

> 你可以像这样在Markdown单元格中使用引用符号`>`，  
以及在代码单元格中使用注释来说明你的实验。  

---

In [1]:
import pickle
import numpy as np

导入HMM参数，初始化所需的起始概率矩阵、转移概率矩阵、发射概率矩阵。

In [2]:
with open("hmm_parameters.pkl", "rb") as f:
    hmm_parameters = pickle.load(f)

# 非断字（B）为第0行，断字（I）为第1行
# 发射概率矩阵中，词典大小为65536，以汉字的Unicode码点（一个整数值）作为行索引
start_probability = hmm_parameters["start_prob"]  # shape(2,)
trans_matrix = hmm_parameters["trans_mat"]  # shape(2, 2)
emission_matrix = hmm_parameters["emission_mat"]  # shape(2, 65536)

定义待处理的句子：

In [3]:
# TODO: 将your_name中的xxx替换为你的姓名【1分】
your_name = "刘翰文"
input_sentence = f"{your_name}是一名优秀的学生"

实现Viterbi算法，并以此进行中文分词。

In [4]:
def viterbi(sent_orig: str, start_prob: np.ndarray, trans_mat: np.ndarray, emission_mat: np.ndarray) -> str:
    """
    Viterbi算法进行中文分词。

    Args:
        sent_orig: str - 输入的句子
        start_prob: numpy.ndarray - 起始概率矩阵
        trans_mat: numpy.ndarray - 转移概率矩阵
        emission_mat: numpy.ndarray - 发射概率矩阵

    Return:
        str - 中文分词的结果
    """
    
    #  将汉字转为数字表示
    sent_ord = [ord(x) for x in sent_orig]

    # `dp`用来储存不同位置每种标注（B/I）的最大概率值
    dp = np.zeros((2, len(sent_ord)), dtype=float)

    # `path`用来储存最大概率对应的上步B/I选择
    #  例如 path[1][7] == 1 意味着第8个（从1开始计数）字符标注I对应的最大概率，其前一步的隐状态为1（I）
    #  例如 path[0][5] == 1 意味着第6个字符标注B对应的最大概率，其前一步的隐状态为1（I）
    #  例如 path[1][1] == 0 意味着第2个字符标注I对应的最大概率，其前一步的隐状态为0（B）
    path = np.zeros((2, len(sent_ord)), dtype=int)

    #  TODO: 第一个位置的最大概率值计算【1分】
    #   start乘emissioin得到
    dp[0, 0] = start_prob[0] * emission_mat[0, sent_ord[0]]
    dp[1, 0] = start_prob[1] * emission_mat[1, sent_ord[0]]
    
    #  TODO: 其余位置的最大概率值计算（填充dp和path矩阵）【2分】
    #   Viterbi动态规划算法计算dp矩阵并根据前一个0,1的大小关系记录path矩阵
    for i in range(1, len(sent_ord)):
        for j in range(2):
            prob_0 = dp[0, i-1] * trans_mat[0, j] * emission_mat[j, sent_ord[i]] #对于dp[j, i], 计算i-1属于0的概率
            prob_1 = dp[1, i-1] * trans_mat[1, j] * emission_mat[j, sent_ord[i]] #对于dp[j, i], 计算i-1属于1的概率
            if prob_0 >= prob_1:
                dp[j, i] = prob_0
                path[j, i] = 0
            else:
                dp[j, i] = prob_1
                path[j, i] = 1 
                
    #  `labels`用来储存每个位置最有可能的隐状态
    labels = [0 for _ in range(len(sent_ord))]

    #  TODO: 计算labels每个位置上的值（填充labels矩阵）【1分】
    #   从尾开始用path数组进行溯源，使用flag记录上一个是0还是1
    if dp[0, -1] >= dp[1, -1]:
        flag = 0
    else:
        flag = 1
    
    for i in range(len(sent_ord)):
        labels[len(sent_ord)-1-i] = flag
        flag = path[flag, len(sent_ord)-i-1]
    
    #  根据lalels生成切分好的字符串
    sent_split = []
    for idx, label in enumerate(labels):
        if label == 1:
            sent_split += [sent_ord[idx], ord("/")]
        else:
            sent_split += [sent_ord[idx]]
    sent_split_str = "".join([chr(x) for x in sent_split])

    return sent_split_str

In [5]:
print("Viterbi算法分词结果：", viterbi(input_sentence, start_probability, trans_matrix, emission_matrix))

Viterbi算法分词结果： 刘翰/文是/一名/优秀/的/学生/


实现前向算法，计算该句子的概率值。

In [6]:
def compute_prob_by_forward(sent_orig: str, start_prob: np.ndarray, trans_mat: np.ndarray, emission_mat: np.ndarray) -> float:
    """
    前向算法，计算输入中文句子的概率值。

    Args:
        sent_orig: str - 输入的句子
        start_prob: numpy.ndarray - 起始概率矩阵
        trans_mat: numpy.ndarray - 转移概率矩阵
        emission_mat: numpy.ndarray - 发射概率矩阵

    Return:
        float - 概率值
    """

    #  将汉字转为数字表示
    sent_ord = [ord(x) for x in sent_orig]

    # `dp`用来储存不同位置每种隐状态（B/I）下，到该位置为止的句子的概率
    dp = np.zeros((2, len(sent_ord)), dtype=float)

    # TODO: 初始位置概率的计算【1分】
    #   同Viterbi算法
    dp[0, 0] = start_prob[0] * emission_mat[0, sent_ord[0]]
    dp[1, 0] = start_prob[1] * emission_mat[1, sent_ord[0]]

    # TODO: 先计算其余位置的概率（填充dp矩阵），然后返回概率值【1分】
    #   根据前向传播的alpha计算公式计算概率，动态规划填充dp矩阵
    for i in range(1, len(sent_ord)):
        for j in range(2):
            dp[j, i] = dp[0, i-1] * trans_mat[0, j] * emission_mat[j, sent_ord[i]] + dp[1, i-1] * trans_mat[1, j] * emission_mat[j, sent_ord[i]]
    

    return sum([dp[i][len(sent_ord) - 1] for i in range(2)])

实现后向算法，计算该句子的概率值。

In [7]:
def compute_prob_by_backward(sent_orig: str, start_prob: np.ndarray, trans_mat: np.ndarray, emission_mat: np.ndarray) -> float:
    """
    后向算法，计算输入中文句子的概率值。

    Args:
        sent_orig: str - 输入的句子
        start_prob: numpy.ndarray - 起始概率矩阵
        trans_mat: numpy.ndarray - 转移概率矩阵
        emission_mat: numpy.ndarray - 发射概率矩阵

    Return:
        float - 概率值
    """

    #  将汉字转为数字表示
    sent_ord = [ord(x) for x in sent_orig]

    # `dp`用来储存不同位置每种隐状态（B/I）下，从结尾到该位置为止的句子的概率
    dp = np.zeros((2, len(sent_ord)), dtype=float)

    # TODO: 终末位置概率的初始化【1分】
    #   填充1
    dp[:, -1] = 1

    # TODO: 先计算其余位置的概率（填充dp矩阵），然后返回概率值【1分】
    #   根据后向传播算法计算beta值，动态规划填充dp矩阵
    for i in range(1, len(sent_ord)):
        for j in range(2):
            dp[j, len(sent_ord)-i-1] = dp[0, len(sent_ord)-i] * trans_mat[j, 0] * emission_mat[0, sent_ord[len(sent_ord)-i]] + dp[1, len(sent_ord)-i] * trans_mat[j, 1] * emission_mat[1, sent_ord[len(sent_ord)-i]]

    return sum([dp[i][0] * start_prob[i] * emission_mat[i][sent_ord[0]] for i in range(2)])

In [8]:
print("前向算法概率：", compute_prob_by_forward(input_sentence, start_probability, trans_matrix, emission_matrix))
print("后向算法概率：", compute_prob_by_backward(input_sentence, start_probability, trans_matrix, emission_matrix))

前向算法概率： 2.229764946349446e-33
后向算法概率： 2.2297649463494462e-33


### 实验总结
中文分词相当于用B和I对句子进行词性标注，使用2.2提到的Viterbi算法和前后向传播算法的公式可以计算最大的标注概率，即为最可能的分词方法。
在实际情况中，可以看到Viterbi算法无法准确对人名进行划分，可能是由于出现频率太低导致。前向和后向传播对概率的计算得到了一样的概率，即为Viterbi算法得到最可能划分的概率。

## 任务二：BPE算法用于英文分词

任务二评分标准：

1. 共有6处TODO需要填写，每个TODO计1-2分，共9分，预计代码量50行。
2. 允许自行修改、编写代码完成。对于该情况，请补充注释以便于评分，否则结果不正确将导致较多的扣分。
3. 用于说明实验的文字和总结不额外计分，但不写会导致扣分。

> 你可以像这样在Markdown单元格中使用引用符号`>`，  
以及在代码单元格中使用注释来说明你的实验。  

---

In [9]:
import re
from typing import List, Tuple, Dict

构建空格分词器，将语料中的句子以空格切分成单词，然后将单词拆分成字母加`</w>`的形式。例如`apple`将变为`a p p l e </w>`。

In [10]:
_splitor_pattern = re.compile(r"[^a-zA-Z']+|(?=')")
_digit_pattern = re.compile(r"\d+")


def white_space_tokenize(corpus: List[str]) -> List[List[str]]:
    """
    先正则化（字母转小写、数字转为N、除去标点符号），然后以空格分词语料中的句子，例如：  
    输入 `corpus = ["I am happy.", "I have 10 apples!"]`，  
    得到 `[["i", "am", "happy"], ["i", "have", "N", "apples"]]`

    Args:
        corpus: List[str] - 待处理的语料

    Return:
        List[List[str]] - 二维List，内部的List由每个句子的单词str构成
    """

    tokeneds = [list(filter(lambda token: len(token) > 0, _splitor_pattern.split(_digit_pattern.sub("N", sentence.lower())))) for sentence in corpus]

    return tokeneds

编写相应函数构建BPE算法需要用到的初始状态词典。

In [11]:
def build_bpe_vocab(corpus: List[str]) -> Dict[str, int]:
    """
    将语料进行white_space_tokenize处理后，将单词每个字母以空格隔开、结尾加上</w>后，构建带频数的字典，例如：  
    输入 `corpus = ["I am happy.", "I have 10 apples!"]`，  
    得到
    ```python
    {
        'i </w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
    }
    ```

    Args:
        corpus: List[str] - 待处理的语料

    Return:
        Dict[str, int] - "单词分词状态->频数"的词典
    """

    tokenized_corpus = white_space_tokenize(corpus)

    bpe_vocab = dict()

    # TODO: 完成函数体【1分】
    #   分开每个单词并在结尾加上' </w>'，并记录在字典中：无则记录有则加1
    for sentence in tokenized_corpus:
        for word in sentence:
            result = ' '.join(word) + ' </w>'
            if result in bpe_vocab:
                bpe_vocab[result] += 1
            else:
                bpe_vocab[result] = 1

    return bpe_vocab



编写所需的其他函数：

In [12]:
def get_bigram_freq(bpe_vocab: Dict[str, int]) -> Dict[Tuple[str, str], int]:
    """
    统计"单词分词状态->频数"的词典中，各bigram的频次（假设该词典中，各个unigram以空格间隔），例如：  
    输入 
    ```python
    bpe_vocab = {
        'i </w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
    }
    ```
    得到
    ```python
    {
        ('i', '</w>'): 2,
        ('a', 'm'): 1,
        ('m', '</w>'): 1,
        ('h', 'a'): 2,
        ('a', 'p'): 2,
        ('p', 'p'): 2,
        ('p', 'y'): 1,
        ('y', '</w>'): 1,
        ('a', 'v'): 1,
        ('v', 'e'): 1,
        ('e', '</w>'): 1,
        ('N', '</w>'): 1,
        ('p', 'l'): 1,
        ('l', 'e'): 1,
        ('e', 's'): 1,
        ('s', '</w>'): 1
    }
    ```

    Args:
        bpe_vocab: Dict[str, int] - "单词分词状态->频数"的词典

    Return:
        Dict[Tuple[str, str], int] - "bigram->频数"的词典
    """

    bigram_freq = dict()

    # TODO: 完成函数体【1分】
    #   遍历所有的两单词组合看是否在字典，有则加频率无则填入字典
    for word, freq in bpe_vocab.items():
        unigram = word.split(' ')
        for i in range(len(unigram) - 1):
            bigram = (unigram[i], unigram[i + 1])
            if bigram in bigram_freq:
                bigram_freq[bigram] += freq
            else:
                bigram_freq[bigram] = freq
    
    return bigram_freq

In [13]:
def refresh_bpe_vocab_by_merging_bigram(bigram: Tuple[str, str], old_bpe_vocab: Dict[str, int]) -> Dict[str, int]:
    """
    在"单词分词状态->频数"的词典中，合并指定的bigram（即去掉对应的相邻unigram之间的空格），最后返回新的词典，例如：  
    输入 
    ```python
    bigram = ('i', '</w>'), old_bpe_vocab = {
        'i </w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
    }
    ```
    得到
    ```python
    {
        'i</w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
    }
    ```
    
    Args:
        bigram: Tuple[str, str] - 待合并的bigram
        old_bpe_vocab: Dict[str, int] - 初始"单词分词状态->频数"的词典

    Return:
        Dict[str, int] - 合并后的"单词分词状态->频数"的词典
    """

    new_bpe_vocab = dict()
    # TODO: 完成函数体【1分】
    #   得到目标bigram后合并old中的两个对应的unigrams并在new中加上频率
    target = bigram[0] + ' ' + bigram[1]
    for word, freq in old_bpe_vocab.items():
        result = word.replace(target, ''.join(bigram))
        new_bpe_vocab[result] = new_bpe_vocab.get(result, 0) + freq

    return new_bpe_vocab

In [14]:
def get_bpe_tokens(bpe_vocab: Dict[str, int]) -> List[Tuple[str, int]]:
    """
    根据"单词分词状态->频数"的词典，返回所得到的BPE分词列表，并将该列表按照分词长度降序排序返回，例如：  
    输入 
    ```python
    bpe_vocab = {
        'i</w>': 2,
        'a m </w>': 1,
        'ha pp y </w>': 1,
        'ha v e </w>': 1,
        'N </w>': 1,
        'a pp l e s </w>': 1
    }
    ```
    得到
    ```
    [
        ('i</w>', 2),
        ('ha', 2),
        ('pp', 2),
        ('a', 2),
        ('m', 1),
        ('</w>', 5),
        ('y', 1),
        ('v', 1),
        ('e', 2),
        ('N', 1),
        ('l', 1),
        ('s', 1)
    ]
    ```

    Args:
        bpe_vocab: Dict[str, int] - "单词分词状态->频数"的词典

    Return:
        List[Tuple[str, int]] - BPE分词和对应频数组成的List
    """

    #bpe_tokens = []
    bpe_tokens = dict() #将list先用dict替代，用于统计数量，最后通过sorted函数返回一个list
    # TODO: 完成函数体【2分】
    #   先统一根据空格划分token然后填入字典，最后把字典的元素按词长排序，这里'</w>'被看做一个词，用不会出现在单词中的单个符号替代
    for word, freq in bpe_vocab.items():
        token = word.split(' ')
        for t in token:
            if t in bpe_tokens:
                bpe_tokens[t] += freq
            else:
                bpe_tokens[t] = freq
    bpe_tokens = sorted(bpe_tokens.items(), key=lambda x: len(x[0].replace('</w>', '*')), reverse=True)
    return bpe_tokens

In [15]:
def print_bpe_tokenize(word: str, bpe_tokens: List[Tuple[str, int]]):
    """
    根据按长度降序的BPE分词列表，将所给单词进行BPE分词，最后打印结果。
    
    思想是，对于一个待BPE分词的单词，按照长度顺序从列表中寻找BPE分词进行子串匹配，  
    若成功匹配，则对该子串左右的剩余部分递归地进行下一轮匹配，直到剩余部分长度为0，  
    或者剩余部分无法匹配（该部分整体由`"<unknown>"`代替）
    
    例1：  
    输入 `word = "supermarket"`, `bpe_tokens=[
        ("su", 20),
        ("are", 10),
        ("per", 30),
    ]`  
    最终打印 `"su per <unknown>"`

    例2：  
    输入 `word = "shanghai"`, `bpe_tokens=[
        ("hai", 1),
        ("sh", 1),
        ("an", 1),
        ("</w>", 1),
        ("g", 1)
    ]`  
    最终打印 `"sh an g hai </w>"`

    Args:
        word: str - 待分词的单词
        bpe_tokens: List[Tuple(str, int)] - BPE分词和对应频数组成的列表
    """

    # TODO: 请尝试使用递归函数定义该分词过程【2分】
    #   先定义结束情况：sub_word为空，在将sub_word分为token, front, after三部分，front, after继续递归，如果无token，最后返回'<unknown>'
    def bpe_tokenize(sub_word: str) -> str:
        if not sub_word:
            return ''
        for token, freq in bpe_tokens:
            pos = sub_word.find(token)
            if pos != -1:
                front = sub_word[:pos]
                after = sub_word[pos+len(token):]
                return (bpe_tokenize(front).strip() + " " + token + " " + bpe_tokenize(after).strip()).strip()
        return '<unknown>'
    res = bpe_tokenize(word + "</w>")
    print(res)

开始读取数据集并训练BPE分词器：

In [16]:
with open("data/news.2007.en.shuffled.deduped.train", encoding="utf-8") as f:
    training_corpus = list(map(lambda l: l.strip(), f.readlines()[:1000]))

print("Loaded training corpus.")

Loaded training corpus.


In [17]:
training_iter_num = 300

training_bpe_vocab = build_bpe_vocab(training_corpus)
for i in range(training_iter_num):
    # TODO: 完成训练循环内的代码逻辑【2分】
    #   不断找到最频繁的token并进行合并bigram
    bigram_freq = get_bigram_freq(training_bpe_vocab)
    target_bigram = max(bigram_freq, key=lambda x: bigram_freq[x])
    training_bpe_vocab = refresh_bpe_vocab_by_merging_bigram(target_bigram, training_bpe_vocab)

training_bpe_tokens = get_bpe_tokens(training_bpe_vocab)

测试BPE分词器的分词效果：

In [18]:
test_word = "naturallanguageprocessing"

print("naturallanguageprocessing 的分词结果为：")
print_bpe_tokenize(test_word, training_bpe_tokens)

naturallanguageprocessing 的分词结果为：
n atur al lan gu age pro ce s sing</w>


### 实验总结
BPE算法在小规模数据集上进行训练所需的时间非常短，且有快速的推理速度，从结果来看可能具有较好的分割效果